# Wav2Vec2 Transformer

Wav2Vec2 is a speech model that accepts a float array corresponding to the raw waveform of the speech signal.

In this notebook, we will train the Wav2Vec2 base model, built on the Hugging Face Transformers library, to determine the instrument present in an audio segment.

## Import Libraries

In [1]:
from transformers import Wav2Vec2ForSequenceClassification, Wav2Vec2Processor, AdamW
import torch
from torch.utils.data import Dataset, DataLoader, ConcatDataset
import torchaudio
from torchaudio.transforms import Resample
from pathlib import Path

from datetime import datetime
from IPython.display import clear_output

## Set Parameters for Dataset

In [2]:
target_duration = 1 # 1 second

## Define the Dataset

The `AudioDataset` class is designed to handle the loading and preprocessing of audio files for the model.

The only preprocessing done is the resampling of the audio file if its sampling rate is not already 16000Hz.

In [3]:
class AudioDataset(Dataset):
    total_files_loaded = 0  # Class attribute to count files loaded across all instances

    def __init__(self, base_path, class_name, processor, class_names, target_duration=target_duration): # 1 second
        self.base_path = Path(base_path)
        self.class_name = class_name
        self.processor = processor
        self.class_names = class_names
        self.target_duration = target_duration
        self.label_id = self.class_names.index(class_name)  # Get numerical ID
        self.audios = list((self.base_path / self.class_name).glob('*.wav'))
    
    def __len__(self):
        length = len(self.audios)
        print(f"Reported dataset length: {length}")
        return length

    def __getitem__(self, idx):
        if idx >= len(self.audios):
            raise IndexError(f"Index {idx} out of range for dataset with length {len(self.audios)}.")
        audio_path = self.audios[idx]

        AudioDataset.total_files_loaded += 1
        print(f"Status: {AudioDataset.total_files_loaded}/{len(self.audios)*3}\t{self.class_name},\t Accessing index: {idx},\t Audio path: {audio_path}")
        
        waveform, sample_rate = torchaudio.load(audio_path)

        # Preprocessing
        if sample_rate != 16000:
            resampler = Resample(orig_freq=sample_rate, new_freq=16000)
            waveform = resampler(waveform)

        # input feature
        input_values = self.processor(waveform, sampling_rate=16000, return_tensors="pt").input_values
        
        return {"input_values": input_values.squeeze(), "labels": torch.tensor(self.label_id)}

## Initialize Data Loaders

Set up the data loaders required for training and validation.

In [4]:
# Configuration and setup
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
processor = Wav2Vec2Processor.from_pretrained("facebook/wav2vec2-base")

# Paths for datasets
base_train_path = 'solo_train_data/split'
base_val_path = 'solo_test_data/split'

# Classes
class_names = ["Cello", "Piano", "Violin"]

# Instantiate datasets for each class
train_datasets = {}
val_datasets = {}

# Create datasets
for class_name in class_names:
  train_datasets[class_name] = AudioDataset(base_train_path, class_name, processor, class_names)
  val_datasets[class_name] = AudioDataset(base_val_path, class_name, processor, class_names)

# Concatenate datasets into a single dataset for training and validation
train_dataset = ConcatDataset(list(train_datasets.values()))
val_dataset = ConcatDataset(list(val_datasets.values()))

c:\Users\heidi\AppData\Local\Programs\Python\Python39\lib\site-packages\transformers\configuration_utils.py:363: UserWarning: Passing `gradient_checkpointing` to a config initialization is deprecated and will be removed in v5 Transformers. Using `model.gradient_checkpointing_enable()` instead, or if you are using the `Trainer` API, pass `gradient_checkpointing=True` in your `TrainingArguments`.
  warnings.warn(


Reported dataset length: 1597
Reported dataset length: 1597
Reported dataset length: 1597
Reported dataset length: 143
Reported dataset length: 407
Reported dataset length: 92


## Set Parameters for Training

In [ ]:
# Training Parameters
num_epochs = 200
batch_size = 64
num_labels = 3  # Cello, Piano, Violin

## Load the Base Model, Create DataLoader, and Set the Model Optimiser

AdamW is a variant of the Adam optimizer with modifications for better handling of weight decay. <br>
It is necessary in this project because it decouples weight decay from the gradient updates, providing better regularization and often leading to improved training convergence, especially beneficial when fine-tuning pre-trained deep learning models. This optimizer is particularly effective in preventing overfitting and ensuring more stable updates.

In [5]:
# Load the processor and the base model
model = Wav2Vec2ForSequenceClassification.from_pretrained("facebook/wav2vec2-base", num_labels=num_labels)
model.to(device)

# Create DataLoaders
train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True, num_workers=6, pin_memory=True, persistent_workers=True)
val_loader = DataLoader(val_dataset, batch_size=batch_size, num_workers=6, pin_memory=True, persistent_workers=True)

# Optimizer
optimiser = AdamW(model.parameters(), lr=5e-5)

Some weights of Wav2Vec2ForSequenceClassification were not initialized from the model checkpoint at facebook/wav2vec2-base and are newly initialized: ['classifier.bias', 'classifier.weight', 'projector.bias', 'projector.weight', 'wav2vec2.encoder.pos_conv_embed.conv.parametrizations.weight.original0', 'wav2vec2.encoder.pos_conv_embed.conv.parametrizations.weight.original1']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
c:\Users\heidi\AppData\Local\Programs\Python\Python39\lib\site-packages\transformers\optimization.py:521: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(


## Training the Model

Training of 1 epoch takes about 3 hours on a Ryzen 3600 CPU.

If the code for training does not work in the notebook, please use the Python file `model_training`.

In [6]:
best_val_accuracy = 0.0

for epoch in range(num_epochs):
  # Training phase
  model.train()
  for batch in train_loader:
    optimiser.zero_grad()
    input_values = batch["input_values"].to(device)
    labels = batch["labels"].to(device)
    outputs = model(input_values=input_values, labels=labels)
    loss = outputs.loss
    loss.backward()
    optimiser.step()
  
  # Evaluation phase
  model.eval()
  correct_predictions = 0
  total_predictions = 0
  total_loss = 0
  with torch.no_grad():
    for batch in val_loader:
      input_values = batch["input_values"].to(device)
      labels = batch["labels"].to(device)
      outputs = model(input_values=input_values, labels=labels)
      loss = outputs.loss
      total_loss += loss.item()
      
      # Calculate accuracy
      predictions = torch.argmax(outputs.logits, dim=-1)
      correct_predictions += (predictions == labels).sum().item()
      total_predictions += labels.size(0)

  val_accuracy = correct_predictions / total_predictions
  val_loss = total_loss / len(val_loader)

  print(f"Epoch {epoch+1}, Loss: {val_loss:.4f}, Accuracy: {val_accuracy:.4f}")
  
  # Save the model if it has the best accuracy so far
  if val_accuracy > best_val_accuracy:
    best_val_accuracy = val_accuracy
    # Create a directory to save the model, using the current date and time for uniqueness
    current_datetime = datetime.now().strftime('%Y-%m-%d_%H-%M-%S')
    model_save_path = f'model/{current_datetime}_{val_accuracy:.4f}.bin'
    torch.save(model.state_dict(), model_save_path)
    print(f"New best model saved with accuracy: {val_accuracy:.4f} at '{model_save_path}'")